## 2018 Data Import and Cleaning

In [1]:
# import pandas and numpy
import pandas as pd
import numpy as np

### Data Import

In [2]:
# find the path to call to import the files
!ls ../data

act_2017.csv      combined_2017.csv sat_2017.csv
act_2018.csv      final.csv         sat_2018.csv


In [3]:
# assign the file paths variable names
act_2018 = '../data/act_2018.csv'
sat_2018 = '../data/sat_2018.csv'

# read in the 2018 SAT and ACT data files assign them to pandas dataframes
act_18 = pd.read_csv(act_2018)
sat_18 = pd.read_csv(sat_2018)

### ACT 2018 Cleaning

In [4]:
# check out the first few rows of act_18 using the .head() method
act_18.head()

,State,Participation,Composite
0,Maine,7%,24
1,Rhode Island,15%,24.2
2,New Hampshire,16%,25.1
3,Delaware,17%,23.8
4,Pennsylvania,20%,23.5


In [5]:
# check the act_18 data with .describe() method
act_18.describe()

,State,Participation,Composite
count,53,53,53
unique,52,30,40
top,Maine,100%,20
freq,2,17,3


This description shows that there is an unexpected duplicate "Maine" value in the State column.

In [6]:
# locate the double value using a mask
mask = act_18['State'] == 'Maine'
act_18[mask]

,State,Participation,Composite
0,Maine,7%,24
52,Maine,7%,24


In [7]:
# the Maine rows are identical; drop one of them (confirmed against source data)
act_18.drop(52, inplace=True)

In [8]:
# run act_18.describe() again
act_18.describe()

,State,Participation,Composite
count,52,52,52
unique,52,30,40
top,Missouri,100%,23.9
freq,1,17,3


In [9]:
# check act_18 data with .info() method
act_18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 0 to 51
Data columns (total 3 columns):
State            52 non-null object
Participation    52 non-null object
Composite        52 non-null object
dtypes: object(3)
memory usage: 1.6+ KB


'Participation' and 'Composite' columns should have datatypes int and float, respectively; the data needs to be cleaned. 

In [10]:
# define custom participation_to_numeric; previously defined for 2017 notebook
def participation_to_numeric(dataframe, column = 'Participation'):
    dataframe[column] = dataframe[column].map(lambda x: x.replace('%', ''))
    dataframe[column] = dataframe[column].map(lambda x: int(x))

In [11]:
# convert Participation column to numeric using custom participation_to_numeric function
participation_to_numeric(act_18)

# re-check datatypes to make sure the function worked
act_18.dtypes

State            object
Participation     int64
Composite        object
dtype: object

Participation column successfully cleaned and converted to 'int' data.

In [12]:
# act_18['Composite'].astype(float)

I tried to convert act_18 Composite column to float data using the .astype() method, but the data threw a Value error and indicated one of the elements of the series: '#REF!'

- ValueError: could not convert string to float: '#REF!'

In [13]:
# use a mask to find the row that has the indicated error
mask = act_18['Composite'] == '#REF!'
act_18[mask]

,State,Participation,Composite
23,National,50,#REF!


In [14]:
# drop National row, since we dropped it from the 2017 data
# pass in parameter 'inplace=True' so the drop gets saved to the dataframe

act_18.drop(23, inplace=True)

In [15]:
# convert the clean Composite column data to dtype float, using the .astype() method
# and save the change by reassigning act_18['Composite'] to the changed series
act_18['Composite'] = act_18['Composite'].astype(float)

In [16]:
# re-check datatypes
act_18.dtypes

State             object
Participation      int64
Composite        float64
dtype: object

looks good!

In [17]:
# sort act_18 alphabetically by state column using .sort_values() method
act_18 = act_18.sort_values('State')
# and reset the index using the .reset_index() method
# passing in the parameter 'drop=True' to avoid inserting the old index as a column in the resulting dataframe
act_18.reset_index(drop=True, inplace=True)

In [18]:
# use .head() method to check that the dataframe is now sorted by state
# and had the index reset
act_18.head()

,State,Participation,Composite
0,Alabama,100,19.1
1,Alaska,33,20.8
2,Arizona,66,19.2
3,Arkansas,100,19.4
4,California,27,22.7


In [19]:
# rename columns in act_18 so all of the column names are lowercase
act_18.columns = act_18.columns.map(lambda x: x.lower())

# add the prefix '2018_act_' to every column name except 'state'
act_18.columns = act_18.columns.map(lambda x: x if x == 'state' else '2018_act_' + x)

# check that the updates took by calling the .columns attribute
act_18.columns

Index(['state', '2018_act_participation', '2018_act_composite'], dtype='object')

In [20]:
# found out during EDA that, while act_18 state refers to Washington, DC as "Washington, D.C., 
# all of the other dataframes referred to it as "District of Columbia"

# use a mask to locate it
mask = act_18['state'] == 'Washington, D.C.'
act_18[mask]

,state,2018_act_participation,2018_act_composite
47,"Washington, D.C.",32,23.6


In [21]:
# rename it so that it matches the other dataframes
act_18['state'][47] = 'District of Columbia'

/Users/kevinc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
# check to make sure the change occurred
act_18[mask]

,state,2018_act_participation,2018_act_composite
47,District of Columbia,32,23.6


### 2018 SAT Cleaning

In [23]:
# check the first few rows of sat_18 using the .head() method
sat_18.head()

,State,Participation,Evidence-Based Reading and Writing,Math,Total
0,Alabama,6%,595,571,1166
1,Alaska,43%,562,544,1106
2,Arizona,29%,577,572,1149
3,Arkansas,5%,592,576,1169
4,California,60%,540,536,1076


In [24]:
# check the sat_18 data with .describe() method
sat_18.describe()

,Evidence-Based Reading and Writing,Math,Total
count,51.000000,51.000000,51.000000
mean,563.686275,556.235294,1120.019608
std,47.502627,47.772623,94.155083
min,480.000000,480.000000,977.000000
25%,534.500000,522.500000,1057.500000
50%,552.000000,544.000000,1098.000000
75%,610.500000,593.500000,1204.000000
max,643.000000,655.000000,1298.000000


In [25]:
# check sat_18 datatypes using .dtypes attribute
sat_18.dtypes

State                                 object
Participation                         object
Evidence-Based Reading and Writing     int64
Math                                   int64
Total                                  int64
dtype: object

need to convert the Participation column to 'int' data

In [26]:
# convert Participation columns to numeric using custom formula defined above.
participation_to_numeric(sat_18)

# and re-check datatypes to make sure the function worked
sat_18.dtypes

State                                 object
Participation                          int64
Evidence-Based Reading and Writing     int64
Math                                   int64
Total                                  int64
dtype: object

In [27]:
# renaming columns in sat_18 so all column names are lower_case
sat_18.columns = sat_18.columns.map(lambda x: x.lower())

# add the prefix '2018_sat_' to every column name except 'state'
sat_18.columns = sat_18.columns.map(lambda x: x if x == 'state' else '2018_sat_' + x)

# use the .rename() method to rename the column '2018_sat_evidence-based reading and writing'
# so that it has underscores in place of spaces
sat_18.rename(columns={'2018_sat_evidence-based reading and writing':'2018_sat_evidence-based_reading_and_writing' }, inplace=True)
sat_18.columns

Index(['state', '2018_sat_participation',
       '2018_sat_evidence-based_reading_and_writing', '2018_sat_math',
       '2018_sat_total'],
      dtype='object')

#### Merge 2017 and 2018 data into a single dataframe

In [28]:
# import combined 2017 data and assign it to 'combined_2017'
# pass the parameter 'index_col = 0' to prevent .read_csv() method from creating a new index column
combined_2017 = pd.read_csv('../data/combined_2017.csv', index_col=0)

In [29]:
# use the .merge() method to combine act_18 with combined_2017
# and assign it to a temporary variable named 'temp'
temp = combined_2017.merge(act_18, on='state')

# use the .merge() method to combine sat_18 with temp
# and assign it to a variable named 'final'
final = temp.merge(sat_18, on='state')

In [30]:
# use the .columns attribute to check that all expected columns are present
final.columns

Index(['state', '2017_act_participation', '2017_act_english', '2017_act_math',
       '2017_act_reading', '2017_act_science', '2017_act_composite',
       '2017_sat_participation', '2017_sat_evidence-based_reading_and_writing',
       '2017_sat_math', '2017_sat_total', '2018_act_participation',
       '2018_act_composite', '2018_sat_participation',
       '2018_sat_evidence-based_reading_and_writing', '2018_sat_math',
       '2018_sat_total'],
      dtype='object')

In [31]:
# use the .to_csv() method to save the 'final' dataframe
# passing the relative path '../data/final.csv' as the argument
# to save the dataframe as a csv named 'final.csv'
# in project_1's folder 'data'
final.to_csv('../data/final.csv')